In [1]:
from malbecs.preprocess import wine

In [2]:
path = '../../data/raw/UH_2023_TRAIN.txt'

wine_data = wine.load_wine_dataset(path)

In [3]:
wine_data = wine.norm_columns(wine_data)

wine_data.columns

Index(['campana', 'id_finca', 'id_zona', 'id_estacion', 'altitud', 'variedad',
       'modo', 'tipo', 'color', 'superficie', 'produccion'],
      dtype='object')

In [4]:
wine_data = wine.process_altitud(wine_data)

wine_data.head()

,campana,id_finca,id_zona,id_estacion,altitud,variedad,modo,tipo,color,superficie,produccion
0,14,76953,515,4,660.0,26,2,0,1,0.0,22215.0
1,14,84318,515,4,660.0,26,2,0,1,0.0,22215.0
2,14,85579,340,4,520.0,32,2,0,1,0.0,20978.0
3,14,69671,340,4,520.0,32,2,0,1,0.0,40722.0
4,14,14001,852,14,0.0,81,1,0,1,0.0,14126.0


In [5]:
wine_data = wine.replace_zeros_with_na(wine_data, cols=['superficie','altitud'])

wine_data.isnull().sum()

campana           0
id_finca          0
id_zona           0
id_estacion       0
altitud          54
variedad          0
modo              0
tipo              0
color             0
superficie     6518
produccion     1075
dtype: int64

In [41]:
wine_data[wine_data['id_finca']==14843]

wine_data[wine_data['campana']==20].dropna()

,campana,id_finca,id_zona,id_estacion,altitud,variedad,modo,tipo,color,superficie,produccion
6476,20,48626,302,13,600.0,32,2,0,1,3.7503,19511.107
6477,20,47921,302,13,600.0,32,2,0,1,7.3735,38678.893
6478,20,5696,919,14,655.0,59,1,0,1,7.6200,7092.800
6480,20,98814,919,14,655.0,32,2,0,1,7.1657,6660.000
6481,20,98814,919,14,655.0,40,2,0,1,3.5000,6970.000
...,...,...,...,...,...,...,...,...,...,...,...
7477,20,37461,239,6,700.0,52,2,0,1,3.6800,28468.300
7478,20,58769,239,6,700.0,32,2,0,1,4.2500,38480.000
7479,20,58769,239,6,700.0,59,2,0,1,4.1600,39380.000
7480,20,88928,239,6,700.0,40,2,0,1,4.7500,53060.000


,superficie,produccion
0,NaN,22215.0
1,NaN,22215.0
2,NaN,20978.0
3,NaN,40722.0
4,NaN,14126.0
...,...,...
7477,3.680,28468.3
7478,4.250,38480.0
7479,4.160,39380.0
7480,4.750,53060.0


In [37]:
import numpy as np
import pandas as pd
from sklearn.compose import make_column_transformer
import category_encoders as ce
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer, StandardScaler

wine_data[wine_data['id_finca'] == 14843]

cat_cols = ['id_finca', 'id_zona', 'id_estacion', 'variedad','modo','tipo','color']
data = wine_data[wine_data['campana']<22]

X = data.drop(columns=['produccion'])
# .fillna(-1)
y = data['produccion']

wine_num_cols = [c for c in X.columns.to_list() if c not in cat_cols]

X[cat_cols] = X[cat_cols].astype('category')

test_index = X[X['campana']==20].index
train_index = X[X['campana']!=21].index

X_train, X_test, y_train, y_test = X.loc[train_index], X.loc[test_index],y.loc[train_index], y.loc[test_index]

# imputer = KNNImputer()
# X_train['superficie'] = imputer.fit_transform(X = pd.concat([X_train, y_train], axis=1))[:,0]
X_test = X_test.fillna(-1)
X_train = X_train.fillna(-1)
# X_test['superficie'] = 
# imputer.transform(X_test)
# pd.DataFrame(
#     imputer.fit_transform(pd.concat([X_train[['superficie']],y_train],axis=1), y),
#     columns=['superficie','produccion']
# )

prep = make_column_transformer(
    (ce.OrdinalEncoder(), cat_cols),
    (PowerTransformer(), wine_num_cols),
    remainder='passthrough'
)

m = make_pipeline(
    prep,
    RandomForestRegressor(
        n_estimators=100,
        max_features='sqrt',
        criterion='poisson',
        max_depth=8,
        bootstrap=True,
        random_state=42, n_jobs=-1),
)
m.fit(X_train, y_train)

np.sqrt(mean_squared_error(y_test, m.predict(X_test)))

5998.976487729018

In [38]:
X_test['pred'] = m.predict(X_test)
X_test['true'] = y_test
X_test['err'] = np.abs(X_test['pred'] - X_test['true'])
X_test.sort_values('err',ascending=False)

,campana,id_finca,id_zona,id_estacion,altitud,variedad,modo,tipo,color,superficie,pred,true,err
6667,20,14843,845,16,647.5,87,2,0,0,6.2800,41347.258975,86930.00000,45582.741025
7450,20,84804,804,12,500.0,32,2,0,1,17.1833,98312.737549,130950.00000,32637.262451
7045,20,95678,144,15,620.0,52,2,0,1,4.0200,34911.699640,65452.07700,30540.377360
7026,20,76459,510,15,600.0,52,2,0,1,8.5245,66252.614436,96517.92300,30265.308564
6630,20,48877,506,16,610.0,32,2,0,1,12.8460,69671.134931,99360.10500,29688.970069
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7266,20,10117,441,12,462.5,59,2,0,1,0.4060,3902.142519,3911.37054,9.228021
7230,20,27224,134,12,460.0,40,2,0,1,0.8202,4901.978661,4900.00000,1.978661
7113,20,77069,885,12,600.0,59,1,0,1,2.3254,4258.367501,4256.84700,1.520501
7440,20,36122,698,12,490.0,59,1,0,1,0.9164,2979.410849,2980.74000,1.329151


In [10]:
6862.400/9.6708 * 9.6708

# 51708.184/9.6708
# 5346.836249327874


6862.4

In [6]:
wine_data = wine.fillna_by_group(
    wine_data, cols=['superficie'], 
    group=['id_finca','variedad','modo']
    )

wine_data = wine.fillna_by_group(
    wine_data, 
    cols = ['altitud'], 
    group = ['id_estacion'])

wine_data.isnull().sum()

campana           0
id_finca          0
id_zona           0
id_estacion       0
altitud           0
variedad          0
modo              0
tipo              0
color             0
superficie     1806
produccion     1075
dtype: int64

In [7]:
wine_data = wine.get_sup_tot_camp_finca(wine_data)

wine_data = wine.get_sup_tot_finca(wine_data)

wine_data = wine.get_n_var_finca_camp(wine_data)

In [8]:
wine_data = wine.replace_zeros_with_na(
    wine_data, 
    cols=['sup_tot_camp_finca','superficie_total']
)

In [9]:
wine_data.isnull().sum()

campana                  0
id_finca                 0
id_zona                  0
id_estacion              0
altitud                  0
variedad                 0
modo                     0
tipo                     0
color                    0
superficie            1806
produccion            1075
sup_tot_camp_finca    1414
superficie_total       949
n_var_camp_finca         0
dtype: int64

In [10]:
wine_data = wine.fillna_by_value(
    wine_data, 
    cols=['superficie','sup_tot_camp_finca','superficie_total'],
    value=-1
)

In [11]:
wine_data.to_csv('../../data/processed/wine_pro.csv', index=False)

In [13]:
# wine.preproces_wine_data(path)